# BIO_52101_EP: Cell shapes project

## Import all the necessary libraries

## Sample some of the images

## Load the data

## Group the data by phenotype

## Analyse the images for each group

In [ ]:
### Segmentation

In [ ]:
### Identify the average per image

In [ ]:
### Generalize for the phenotype group

## Inhterpret results